In this notebook I am building the logic that converts the raw sql into a sql that can be processed.

Example: 
Convert "'SELECT (U.S. Open Cup) FROM 1-1046170-5 WHERE Regular Season = 4th, atlantic division' (the original sql" to ""

In [1]:
import json
import pandas as pd

In [2]:
path_text2sql_results = '/Users/bleopold/OneDrive/data-analysis/results_from_sqlova/results_dev.jsonl'

path_dev_tables = '/Users/bleopold/OneDrive/data-analysis/vendors/WikiSQL/data/dev.tables.jsonl'

# Get the data I need

In [3]:
# get names of all the tables for which I have data
with open(path_dev_tables, 'r') as json_file:
    dev_tables_raw_file = list(json_file)

tables_with_data = []

for json_str in dev_tables_raw_file:
    result = json.loads(json_str)
    table_id = result["id"]
    
    tables_with_data.append(table_id)

In [4]:
# for all these tables I have a text to sql result
with open(path_text2sql_results, 'r') as json_file:
    text2sql_results_raw_file = list(json_file)

tables_with_text2sql_result = []

for json_str in text2sql_results_raw_file:
    result = json.loads(json_str)
    table_id = result["table_id"]
    
    tables_with_text2sql_result.append(table_id)
    
tables_with_text2sql_result = list(set(tables_with_text2sql_result))

In [5]:
# for all these tables I have data and a text to sql result
"""
get the intersect of tables_with_data and tables_with_text2sql_result
"""
ids_usable_tables = [_ for _ in tables_with_text2sql_result if _ in tables_with_data]

In [30]:
mapping_table_id_table_name = pd.read_csv(
        "../data/01_raw/wiki-sql/schema_infos/mapping_table_id_table_name.csv"
        ,  sep = '\t'
    )

mapping_table_id_table_name

,table_id,table_name
0,1-10015132-9,toronto_raptors_all-time_roster
1,1-10015132-9,toronto_raptors_all-time_roster


# Define functions

In [6]:
def replace_table_id_with_table_name_in_sql(sql, table_id):
    # holds the table_id in one column and the table_name in the other column
    mapping_table_id_table_name = pd.read_csv(
        "../data/01_raw/wiki-sql/schema_infos/mapping_table_id_table_name.csv"
        ,  sep = '\t'
    )
    
    
    # get the table_name using table_id
    table_name = list(mapping_table_id_table_name[
        mapping_table_id_table_name["table_id"]==table_id
    ]["table_name"])[0]
    
    
    
    # replace table_id for table_name in the original sql
    sql_clean_table_name = sql.replace(table_id, table_name)
    
    return sql_clean_table_name

In [7]:
def replace_enclosed_where_conditions_with_quotes(sql, condition_infos):
    """
    example for condition_infos: [[3, 0, '4th, atlantic division']]
    
    place each condition in the array between dobles quotes
    """
    for condition_info in condition_infos:
        condition_value_orig = condition_info[2]
        condition_value_quoted = "'{}'".format(condition_value_orig)
        sql = sql.replace(condition_value_orig, condition_value_quoted)

    return sql

In [32]:
def insert_clean_column_names(sql, condition_infos, table_id):
    mapping_column_names = pd.read_csv(
        "../data/01_raw/wiki-sql/schema_infos/information_schema.csv"
        ,  sep = '\t'
    )
    
    for condition_info in condition_infos:
        column_order = condition_info[0]
    
    
        column_name_original  = list(
            mapping_column_names[
                (mapping_column_names["table_id"] == table_id)
                &
                (mapping_column_names["column_order"] == column_order)
            ]["column_name_original"]
        )[0]

        column_name_clean  = list(
            mapping_column_names[
                (mapping_column_names["table_id"] == table_id)
                &
                (mapping_column_names["column_order"] == column_order)
            ]["column_name_clean"]
        )[0]


        sql = sql.replace(column_name_original, column_name_clean)
        
        
    return sql

# Convert original sql to parse-able sql

In [42]:
# get one sql/table for testing
with open(path_text2sql_results, 'r') as json_file:
    tables = list(json_file)

#
for table in tables:
    result = json.loads(table)
    
    table_id = result["table_id"]
    
    if table_id in (
                    # '1-1046170-5'
                    # ,'1-1061075-1'
                    '1-10015132-9'
                    # '1-10015132-11'
                    # ,'1-10026563-1'
                    # ,'1-10295819-2'
                    # ,'1-10429820-13'
                   ):
        table_file_path = '../data/01_raw/wiki-sql/tables/{}.csv'.format(table_id)
        text_question = result["nlu"]
        sql_original = result["sql"]
        
        sql = replace_table_id_with_table_name_in_sql(sql_original, table_id)
        sql = replace_original_select_with_clean(sql, result, table_id)
        
        condition_infos = result["query"].get("conds")
        if condition_infos:
            sql = replace_enclosed_where_conditions_with_quotes(sql, condition_infos)
            sql = insert_clean_column_names(sql, condition_infos, table_id)
        
        print(sql)
        print('\n')

{'query': {'agg': 0, 'sel': 0, 'conds': [[5, 0, 'westchester high school']]}, 'table_id': '1-10015132-9', 'nlu': 'Who are all of the players on the Westchester High School club team?', 'sql': 'SELECT (Player) FROM 1-10015132-9 WHERE School/Club Team = westchester high school'}
SELECT (player) FROM toronto_raptors_all-time_roster WHERE schoolclub_team = 'westchester high school'


{'query': {'agg': 0, 'sel': 5, 'conds': [[0, 0, 'amir johnson']]}, 'table_id': '1-10015132-9', 'nlu': 'What school/club team is Amir Johnson on?', 'sql': 'SELECT (School/Club Team) FROM 1-10015132-9 WHERE Player = amir johnson'}
SELECT (schoolclub_team) FROM toronto_raptors_all-time_roster WHERE player = 'amir johnson'


{'query': {'agg': 3, 'sel': 1, 'conds': [[4, 0, '2005-06']]}, 'table_id': '1-10015132-9', 'nlu': 'What are the total amount of numbers on the Toronto team in 2005-06?', 'sql': 'SELECT count(No.) FROM 1-10015132-9 WHERE Years in Toronto = 2005-06'}
SELECT count(no) FROM toronto_raptors_all-time

In [40]:
def replace_original_select_with_clean(sql, result, table_id):
    inf_schema = pd.read_csv(
        "../data/01_raw/wiki-sql/schema_infos/information_schema.csv"
        , sep='\t'
    )
    
    # @todo: can there be more than 1 column?
    select_column_index = result["query"].get('sel')
    
    
    column_name_clean = list(
        inf_schema[
            (inf_schema["table_id"] == table_id)
            &
            (inf_schema["column_order"] == select_column_index)
        ]["column_name_clean"]
    )[0]


    column_name_original = list(
            inf_schema[
                (inf_schema["table_id"] == table_id)
                &
                (inf_schema["column_order"] == select_column_index)
            ]["column_name_original"]
        )[0]


    sql = sql.replace(column_name_original, column_name_clean)
    return sql

# Probleme
* Query selber bauen? replacing ist gefährlich
* toronto_raptors_all-time_roster nicht korrekt slugified
* selects dürfen nicht in klammer sein + müssen slugified sein --> ACHTUNG: darf kein einfaches replace sein nur auf den spaltennamen, weil dder spaltennamen in der tabelle auch stehen könnte --> replace auf inkl "select" + das aggregation (zB count)
* Probleme mit case-sensitive